#### Importing Required Libraries

In [24]:
import json 

from tqdm.auto import tqdm
from dotenv import load_dotenv

from elasticsearch import Elasticsearch
from langchain_ollama import ChatOllama
from langchain_mistralai.chat_models import ChatMistralAI

load_dotenv()

True

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)    

In [3]:
documents = [] 

# looping on all courses
for course_dict in docs_raw:
    # looping on all documents for each course 
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

#### Setting up Elastic Search Client

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [6]:
es_client.info()

ObjectApiResponse({'name': '99d13088ca2c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hqsFvuYCSt288q8Qe_9bFg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                    "text": {"type": "text"},
                    "section": {"type": "text"},
                    "question": {"type": "text"},
                    "course": {"type": "keyword"} 
        }
     }
}


index_name = "faq-index"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq-index'})

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [01:00<00:00, 15.62it/s]


In [11]:
query ="the course hass already started, can I still enroll ?"

In [ ]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es_client.search(index=index_name,
                 body=search_query)

In [15]:
response

ObjectApiResponse({'took': 20, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 399, 'relation': 'eq'}, 'max_score': 48.763668, 'hits': [{'_index': 'faq-index', '_id': 'lfZHdJcBMV1ExLThtNdt', '_score': 48.763668, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'faq-index', '_id': 'l_ZHdJcBMV1ExLThtdcC', '_score': 36.514423, '_source': {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if 

In [19]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [20]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [21]:
def elastic_search(query):
    # Results List
    result_docs = []
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,
                 body=search_query)
    

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [22]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [25]:
def build_prompt(query, retrieved_docs):
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output I don't Know.

    QUESTION: {question}

    CONTEXT: {context}

    ANSWER:
    """.strip()
    
    context = "" 

    for doc in retrieved_docs:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt
    

In [26]:
def get_llm(model_name):
    if model_name == "mistral":
        llm = ChatMistralAI()
    elif model_name == "llama":
        llm = ChatOllama(model="llama3.2-3b") 
    else:
        print("Invalid model name...")
    return llm

In [27]:
def get_llm_response(prompt, model_name="mistral"): 
    llm = get_llm(model_name=model_name)
    response = llm.invoke(prompt)
    return response.content

In [28]:
def rag(query):
    # 1: Retrieval step: Using Elastic Search
    retrieved_docs = elastic_search(query)

    # 2. Augmentation step
    prompt = build_prompt(query, retrieved_docs)
    
    # 3. Generation Step
    answer = get_llm_response(prompt)
    
    return answer

In [30]:
answer = rag(query="the course has already started, can I still enroll?")

answer

"Yes, you can still join the course after it has started. According to the provided context, even if you don't officially enroll, you are still eligible to submit homeworks, but be aware of the deadlines for turning in final projects. It's recommended to manage your time wisely and not leave everything for the last minute."